### Import Required Libraries

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql.functions import from_json, col

### Create Spark Session

In [ ]:
spark = SparkSession.builder \
    .appName("KafkaSparkStreaming") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.2") \
    .getOrCreate()

### Read Data from Kafka Topic

In [ ]:
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "kafka:9092") \
    .option("subscribe", "icu_topic") \
    .load()

### Extract and Process the Kafka Value Column

In [ ]:
processed_df = df.selectExpr("CAST(value AS STRING)")

processed_df = processed_df.withColumnRenamed("value", "data")

### Write Processed Data to Console

In [ ]:
query = processed_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

### Await Termination

In [ ]:
query.awaitTermination()